In [2]:
# ── CONFIG ────────────────────────────────────────────────────────────────────
IN_CSV   = "D:\Digital_Analytics\Cleaned data\Cleaned_Nov.csv"   # adjust to your file
OUT_DIR  = "D:\Digital_Analytics\Star_schema_tables"                       # where to write the star-CSV files

import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 100)

df = pd.read_csv(IN_CSV, parse_dates=["event_time"])
print(f"Loaded {len(df):,} rows")


Loaded 99,898 rows


In [8]:
print(df.columns)             # make sure the column is really called 'timestamp'
print(df['event_time'].dtype)  # see what pandas thinks it is


Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session',
       'is_purchase', 'hour_of_day', 'day_of_week', 'is_weekend'],
      dtype='object')
datetime64[ns, UTC]


In [12]:
# ---- robust dim_date build (works with datetime64[ns, UTC]) ------------------

# 1️⃣  Make sure event_time is datetime64
df["event_time"] = pd.to_datetime(df["event_time"], errors="coerce")

# 2️⃣  Floor to midnight so we keep datetime64 dtype
dates = df["event_time"].dt.floor("D")          # keeps timezone info if any

# 3️⃣  Create dimension
dim_date = (
    pd.DataFrame({"date": dates})
      .drop_duplicates("date")
      .assign(
          date_sk    = lambda x: x["date"].dt.strftime("%Y%m%d").astype(int),
          year       = lambda x: x["date"].dt.year,
          month      = lambda x: x["date"].dt.month,
          day_of_week= lambda x: x["date"].dt.dayofweek,
          is_weekend = lambda x: x["date"].dt.dayofweek.isin([5, 6])
      )
      .sort_values("date")
)

# 4️⃣  Save to CSV ready for BigQuery LOAD
dim_date.to_csv(f"{OUT_DIR}/dim_date.csv", index=False)
print("dim_date rows:", len(dim_date))


dim_date rows: 1


In [13]:
dim_session = (
    df.groupby("user_session")["event_time"]
      .agg(session_start = "min",
           session_end   = "max",
           n_events      = "count")
      .assign(duration_s=lambda x: (x["session_end"]-x["session_start"])
                                   .dt.total_seconds())
      .reset_index()
      .rename(columns={"user_session": "session_sk"})
)
dim_session.to_csv(f"{OUT_DIR}/dim_session.csv", index=False)


In [14]:
dim_product = (
    df[["product_id", "category_id", "category_code", "brand"]]
      .drop_duplicates()
      .rename(columns={"product_id": "product_sk"})
)
dim_product.to_csv(f"{OUT_DIR}/dim_product.csv", index=False)


In [21]:
# ─── Build dim_user with unique synthetic names ──────────────────────────────
import pandas as pd
from faker import Faker
fake = Faker()

# 1️⃣  Get distinct user IDs
dim_user = (
    df[["user_id"]]
      .drop_duplicates()
      .rename(columns={"user_id": "user_sk"})
      .sort_values("user_sk")
      .reset_index(drop=True)
)

# 2️⃣  Generate UNIQUE fake names
fake.unique.clear()                       # reset uniqueness pool
dim_user["user_name"] = [
    fake.unique.name() for _ in range(len(dim_user))
]

# 3️⃣  Save to CSV
out_path = f"{OUT_DIR}/dim_user.csv"
dim_user.to_csv(out_path, index=False)
print(f"Wrote {len(dim_user):,} rows with unique names → {out_path}")


Wrote 22,443 rows with unique names → D:\Digital_Analytics\Star_schema_tables/dim_user.csv


In [18]:
# 1️⃣  Build dim_event_type WITH a numeric surrogate key
codes, uniques = pd.factorize(df["event_type"], sort=True)

dim_event_type = (
    pd.DataFrame({"event_type": uniques})
      .reset_index()
      .rename(columns={"index": "event_type_sk"})   # 0,1,2… become surrogate key
)

# 2️⃣  Add that key back onto the main dataframe
df["event_type_sk"] = codes     # codes array aligns 1-to-1 with df rows

# 3️⃣  Save the dimension
dim_event_type.to_csv(f"{OUT_DIR}/dim_event_type.csv", index=False)


In [19]:
# map date → date_sk
date_map = dict(zip(dim_date["date"], dim_date["date_sk"]))
df["date_sk"] = df["event_time"].dt.floor("D").map(date_map)

fact_events = df[[
    "user_session",         # session_sk  (FK)
    "user_id",              # user_sk     (FK)
    "product_id",           # product_sk  (FK)
    "event_type_sk",        # FK → dim_event_type
    "date_sk",              # FK → dim_date
    "price",
    "is_purchase"
]].rename(columns={
    "user_session": "session_sk",
    "user_id":      "user_sk",
    "product_id":   "product_sk"
})


fact_events.to_csv(f"{OUT_DIR}/fact_events.csv", index=False)
print("fact_events rows:", len(fact_events))


fact_events rows: 99898
